In [ ]:
#### GENERAL ####

# Alow changes to the PlatoSim code outside this notebook
%load_ext autoreload
%autoreload 2

# To interact with the plot use
%matplotlib notebook

import os
import numpy as np
import matplotlib.pyplot as plt

# PlatoSim
import platosim.referenceFrames as rf
from platosim.utilities    import getFunctions
from platosim.simulation   import Simulation
from platosim.simfile      import SimFile
from platosim.matplotlibrc import setup_notebook
setup_notebook()

# Inputs
outputFileName = "16CygA_test"
outputDir=os.getcwd()
sim = Simulation(outputFileName, outputDir=os.getcwd())

In [ ]:
sim["ObservingParameters/RApointing"]   = 83.0
sim["ObservingParameters/DecPointing"]  = 13.0
sim["Platform/SolarPanelOrientation"]   = 180.0

sim["SubField/ZeroPointRow"]    = 3000
sim["SubField/ZeroPointColumn"] = 3000
sim["SubField/NumColumns"]      = 50
sim["SubField/NumRows"]         = 50

sim["Sky/SkyBackground"]    = -1

# Define catalogue
row = np.array([28.0]) + sim["SubField/ZeroPointRow"]
col = np.array([28.0]) + sim["SubField/ZeroPointColumn"]
mag = np.array([6.20])
ID  = [0]

# Automatic catalogue file creation
starcatFile = outputDir + "/16cyg_example.txt"
sim.createStarCatalogFileFromPixelCoordinates(row, col, mag, ID, starcatFile)
row, col

# Make sure the simulation object uses this star catalog
sim["ObservingParameters/StarCatalogFile"] = starcatFile

In [ ]:
photometryFile = os.getcwd() + "/photometry.txt"
sim.createPhotometryTargetFile(ID, photometryFile)

sim["ObservingParameters/NumExposures"] = 100
sim["Photometry/IncludePhotometry"]     = True

In [ ]:
# Run the simulation
simfile = sim.run(removeOutputFile=True)

In [ ]:
f = SimFile("16CygA_test" + ".hdf5")

f.reload()

imageNr = 0

In [ ]:
#image      = f.getImage(imageNr)
#numRows    = f.getInputParameter("SubField", "NumRows")
#numColumns = f.getInputParameter("SubField", "NumColumns")

#fig, ax = f.showImage(imageNr, clipPercentile=1, imgScale="clip",
                      #figsize=(8,8), fontSize=15, useTitle=False,
                      #showStarPositions=True, showStarIDs=False,
                      #colorMap="magma", colorBar=True, showGrid=True) 

In [ ]:
from platosim.h5 import h5get, h5ls

import h5py
h5file = h5py.File("16CygA_test" + ".hdf5", 'r')

In [ ]:
#h5ls(h5file)

In [ ]:
est_flux = h5get(h5file,'Photometry/Lightcurves/starID0/estimatedFlux' )
observations = np.linspace(0,100,100)

In [ ]:
#plt.plot(observations, est_flux)
#plt.show()

In [ ]:
missionDuration = 6.5 * 24 * 60 * 60 ## seconds
missionDuration/25

samplingRate = 100/25
N = 100

yf = np.abs(np.fft.fft(est_flux)[0:50])
xf = np.fft.fftfreq(N, 1/samplingRate)[0:50]

In [ ]:
fig = plt.subplots(nrows=1, ncols=1, dpi=120, figsize=(5,4))

plt.plot(xf, yf, linewidth=0.5)
plt.yscale('log')
plt.yticks(fontsize=10)
plt.xticks(fontsize=10)


plt.ylabel('Amplitude', size=12)
plt.xlabel('Frequency [µHz]', size=12)

plt.show()